In [4]:
import requests
from bs4 import BeautifulSoup
import re
import os
import pandas as pd
from lxml import etree
from datetime import datetime
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
current_directory = os.getcwd()

In [5]:
def normalize_date(text):
    # Biểu thức chính quy để bắt ngày, tháng, năm
    pattern = r"(\d{1,2})\s*tháng\s*(\d{1,2})\s*năm\s*(\d{4})"
    match = re.search(pattern, text)
    
    if match:
        # Lấy ra các thành phần ngày, tháng, năm
        day, month, year = match.groups()
        
        # Chuyển đổi thành định dạng dd-mm-yyyy
        date_obj = datetime.strptime(f"{day}-{month}-{year}", "%d-%m-%Y")
        return date_obj.strftime("%d-%m-%Y")
    return None

In [6]:
def part_text(part,start,end):
    part_list = []
    for i in part[start:end]:
        part_list.append(i.get_text(strip=True).replace('\r', '').replace('\n', '').strip())
        part_text = '\n'.join(part_list)
    return part_text
        

In [22]:
def get_phanloai_link_info(link):
    response2 = requests.get(link.strip())
    soup = BeautifulSoup(response2.text, 'html.parser')
    content1 = soup.find('div', class_='content1')
    tables = content1.find_all('table')

    # Khởi tạo các biến trước để tránh lỗi khi không tìm thấy
    số = "Không tìm thấy 'Số'"
    ngày = "Không tìm thấy 'Ngày'"
    part_1_text = "Không tìm thấy part 1"
    part_2_text = "Không tìm thấy part 2"
    part_3_text = "Không tìm thấy part 3"
    part_4_text = "Không tìm thấy part 4"
    part_5_text = "Không tìm thấy thông tin phần 5"

    # Lấy số văn bản / ngày
    try:
        paragraphs = tables[0].find_all('p')  # Tìm tất cả các thẻ <p> trong bảng đầu tiên
        for para in paragraphs:
            text = para.get_text(strip=True).replace('\r', '').replace('\n', '').strip()
            if 'Số:' in text:
                số = text.replace("Số:", "").strip()
                break  # Ngừng vòng lặp sau khi tìm thấy đoạn văn bản chứa 'Số'
    except AttributeError as e:
        print(f"Lỗi khi tìm 'Số': {e}")

    try:
        paragraphs = tables[0].find_all('p')  # Tìm tất cả các thẻ <p> trong bảng đầu tiên
        for para in paragraphs:
            text = para.get_text(strip=True).replace('\r', '').replace('\n', '').strip()
            if 'ngày' in text or 'Ngày' in text:
                ngày = normalize_date(text)
                break  # Ngừng vòng lặp sau khi tìm thấy đoạn văn bản chứa 'ngày'
    except AttributeError as e:
        print(f"Lỗi khi tìm 'ngày': {e}")

    # Lấy part 1 - bảng đầu của trang
    part1 = tables[1].find_all('p')       
    part1_1_pattern = re.compile(r"1\.\s*[Tt]ên\s*[Hh]àng")
    part1_2_pattern = re.compile(r"2\.\s*Đơn\s*vị")
    part1_3_pattern = re.compile(r"3\.\s*Số")
    part1_3_pattern_2 = re.compile(r"3\.\s*Tờ\s*khai")
    part1_4_pattern = re.compile(r"4\.\s*Tóm\s*tắt")
    part1_5_pattern = re.compile(r"5\.\s*Kết\s*quả")

    # Tìm index các phần
    part1_1_index = next((i for i, p_tag in enumerate(part1) if part1_1_pattern.search(p_tag.get_text(strip=True))), None)
    part1_2_index = next((i for i, p_tag in enumerate(part1) if part1_2_pattern.search(p_tag.get_text(strip=True))), None)
    part1_3_index = next((i for i, p_tag in enumerate(part1) if part1_3_pattern.search(p_tag.get_text(strip=True)) or part1_3_pattern_2.search(p_tag.get_text(strip=True))), None)
    part1_4_index = next((i for i, p_tag in enumerate(part1) if part1_4_pattern.search(p_tag.get_text(strip=True))), None)
    part1_5_index = next((i for i, p_tag in enumerate(part1) if part1_5_pattern.search(p_tag.get_text(strip=True))), None)

    # Kiểm tra nếu các chỉ mục có tồn tại, nếu không gán giá trị thông báo không tìm thấy
    if part1_1_index is not None and part1_2_index is not None:
        part_1_text = part_text(part1, part1_1_index, part1_2_index)
    if part1_2_index is not None and part1_3_index is not None:
        part_2_text = part_text(part1, part1_2_index, part1_3_index)
    if part1_3_index is not None and part1_4_index is not None:
        part_3_text = part_text(part1, part1_3_index, part1_4_index)
    if part1_4_index is not None and part1_5_index is not None:
        part_4_text = part_text(part1, part1_4_index, part1_5_index)

    # Lấy thông tin phần 5 nếu tìm thấy index
    if part1_5_index is not None:
        pattern = re.compile(r'\d{4}\s*\.\s*\d{2}\s*\.\s*\d{2}')
        for p_tag in part1[part1_5_index:]:
            text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
            matches = pattern.findall(text)  # Tìm tất cả các dãy số khớp với pattern trong văn bản
            if matches:
                part_5_text = matches[-1]  # Lấy kết quả cuối cùng khớp với pattern

    # Tạo danh sách chứa thông tin
    info_list = [link.strip(), số, ngày, part_1_text, part_2_text, part_3_text, part_4_text, part_5_text]
    print(link)
    print(info_list)
    return info_list


In [67]:
link = "https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-13850-TB-TCHQ-2014-ket-qua-phan-loai-Thuoc-mau-vo-co-tu-dioxit-titan-xuat-nhap-khau-258404.aspx"

In [68]:
get_phanloai_link_info(link)

['https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-13850-TB-TCHQ-2014-ket-qua-phan-loai-Thuoc-mau-vo-co-tu-dioxit-titan-xuat-nhap-khau-258404.aspx',
 '13850/TB-TCHQ',
 '17-11-2014',
 'Không tìm thấy part 1',
 '2. Đơn vị nhập khẩu: Công ty TNHH IL  Sam Việt Nam; Đ/c: Đường 3, KCN Long Thanh; MST: 3600855877',
 '3. Số, ngày tờ khai hải quan:10015727423/A12 ngày 06/10/2014 đăng ký tại Chi cục HQ Long Thành - Cục Hải  quan Đồng Nai',
 '4. Tóm tắt mô tả và đặc tính hàng hóa:  Thuốc màu vô cơ đi từ titan điôxit, hàm lượng titan điôxit 97,6%, dạng bột,  đã xử lý bề mặt',
 '3206.11.10']

In [37]:
response2 = requests.get(link.strip())
soup = BeautifulSoup(response2.text, 'html.parser')
content1 = soup.find('div', class_='content1')
tables = content1.find_all('table')

# Khởi tạo các biến trước để tránh lỗi khi không tìm thấy
số = "Không tìm thấy 'Số'"
ngày = "Không tìm thấy 'Ngày'"
part_1_text = "Không tìm thấy part 1"
part_2_text = "Không tìm thấy part 2"
part_3_text = "Không tìm thấy part 3"
part_4_text = "Không tìm thấy part 4"
part_5_text = "Không tìm thấy thông tin phần 5"

# Lấy số văn bản / ngày
try:
    paragraphs = tables[0].find_all('p')  # Tìm tất cả các thẻ <p> trong bảng đầu tiên
    for para in paragraphs:
        text = para.get_text(strip=True).replace('\r', '').replace('\n', '').strip()
        if 'Số:' in text:
            số = text.replace("Số:", "").strip()
            break  # Ngừng vòng lặp sau khi tìm thấy đoạn văn bản chứa 'Số'
except AttributeError as e:
    print(f"Lỗi khi tìm 'Số': {e}")

try:
    paragraphs = tables[0].find_all('p')  # Tìm tất cả các thẻ <p> trong bảng đầu tiên
    for para in paragraphs:
        text = para.get_text(strip=True).replace('\r', '').replace('\n', '').strip()
        if 'ngày' in text or 'Ngày' in text:
            ngày = normalize_date(text)
            break  # Ngừng vòng lặp sau khi tìm thấy đoạn văn bản chứa 'ngày'
except AttributeError as e:
    print(f"Lỗi khi tìm 'ngày': {e}")

# Lấy part 1 - bảng đầu của trang
part1 = tables[1].find_all('p')       
part1_1_pattern = re.compile(r"1\.\s*[Tt]ên\s*[Hh]àng")
part1_2_pattern = re.compile(r"2\.\s*Đơn\s*vị")
part1_3_pattern = re.compile(r"3\.\s*Số")
part1_3_pattern_2 = re.compile(r"3\.\s*Tờ\s*khai")
part1_4_pattern = re.compile(r"4\.\s*Tóm\s*tắt")
part1_5_pattern = re.compile(r"5\.\s*Kết\s*quả")


In [38]:
part1

[<p style="margin-bottom:6.0pt"><b>1. Tên hàng theo khai báo</b>: Phối liệu để
   nấu thủy tinh men Ceramic Frit UF- T39</p>,
 <p style="margin-bottom:6.0pt"><b>2. Đơn vị nhập khẩu:</b> Công ty cổ phần
   Gạch men V.T.C; Địa chỉ: Lô 9, Khu công nghiệp Gò Dầu, Huyện Long Thành, Tỉnh
   Đồng Nai; Mã số thuế: 3600723398</p>,
 <p style="margin-bottom:6.0pt"><b>3. Số, ngày tờ khai hải quan</b>:
   10010414600/A12 ngày 20/8/2014 tại Chi cục Hải quan Long Thành - Cục Hải quan
   Tỉnh Đồng Nai</p>,
 <p style="margin-bottom:6.0pt"><b>4, Tóm tắt mô tả và đặc tính hàng hóa</b>:
   Men đã được Frit hóa dạng mảnh vỡ thủy tinh, thành phần gồm SiO<sub>2</sub>
   51,2%, CaO 23,4%, TiO<sub>2</sub> 17,1%, Al<sub>2</sub>O<sub>3</sub> 3.4 %Na<sub>2</sub>O
   2,4%, K<sub>2</sub>O 1,2%, ZrO<sub>2</sub> 0,1%, MgO 0,8%, Fe<sub>2</sub>O<sub>3</sub>
   0,4%</p>,
 <p style="margin-bottom:6.0pt"><b><span lang="EN-GB">5. Kết quả phân loại:</span></b></p>,
 <p style="margin-bottom:6.0pt">Tên thương mại: Ceramic Fri

In [39]:
part1_3_pattern = re.compile(r"3\.\s*Số")
part1_3_pattern_2 = re.compile(r"3\.\s*Tờ\s*khai")
for p_tag in part1:
    text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
    print(text)
    matches = part1_3_pattern.findall(text) or part1_3_pattern_2.findall(text) # Tìm tất cả các dãy số khớp với pattern trong văn bản
    print(matches)
    if matches:
        part1_3_index = part1.index(p_tag)
        print(part1_3_index)

1. Tên hàng theo khai báo: Phối liệu để
  nấu thủy tinh men Ceramic Frit UF- T39
[]
2. Đơn vị nhập khẩu:Công ty cổ phần
  Gạch men V.T.C; Địa chỉ: Lô 9, Khu công nghiệp Gò Dầu, Huyện Long Thành, Tỉnh
  Đồng Nai; Mã số thuế: 3600723398
[]
3. Số, ngày tờ khai hải quan:
  10010414600/A12 ngày 20/8/2014 tại Chi cục Hải quan Long Thành - Cục Hải quan
  Tỉnh Đồng Nai
['3. Số']
2
4, Tóm tắt mô tả và đặc tính hàng hóa:
  Men đã được Frit hóa dạng mảnh vỡ thủy tinh, thành phần gồm SiO251,2%, CaO 23,4%, TiO217,1%, Al2O33.4 %Na2O
  2,4%, K2O 1,2%, ZrO20,1%, MgO 0,8%, Fe2O30,4%
[]
5. Kết quả phân loại:
[]
Tên thương mại: Ceramic Frit
[]
Tên gọi theo cấu tạo, công dụng: Men đã được
  Frit hóa dạng mảnh vỡ thủy tinh, thành phần gồm SiO251,2%, CaO
  23,4%, TiO217,1%, Al2O33.4% Na2O
  2,4%, K2O 1,2%, ZrO20,1%, MgO 0,8%, Fe2O30,4%
[]
Ký, mã hiệu, chủng loại: Không có thông
  tin            
  Nhà sản xuất: Không có thông tin
[]
thuộc nhóm3207"Thuốc màu đã
  pha chế, các chất cản quang đã pha chế và các

In [40]:
part1_4_pattern = re.compile(r"4\.\s*Tóm\s*tắt")
for p_tag in part1:
    text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
    print(text)
    matches = part1_4_pattern.findall(text)  # Tìm tất cả các dãy số khớp với pattern trong văn bản
    print(matches)
    if matches:
        part1_4_index = part1.index(p_tag)
        print(part1_4_index)

1. Tên hàng theo khai báo: Phối liệu để
  nấu thủy tinh men Ceramic Frit UF- T39
[]
2. Đơn vị nhập khẩu:Công ty cổ phần
  Gạch men V.T.C; Địa chỉ: Lô 9, Khu công nghiệp Gò Dầu, Huyện Long Thành, Tỉnh
  Đồng Nai; Mã số thuế: 3600723398
[]
3. Số, ngày tờ khai hải quan:
  10010414600/A12 ngày 20/8/2014 tại Chi cục Hải quan Long Thành - Cục Hải quan
  Tỉnh Đồng Nai
[]
4, Tóm tắt mô tả và đặc tính hàng hóa:
  Men đã được Frit hóa dạng mảnh vỡ thủy tinh, thành phần gồm SiO251,2%, CaO 23,4%, TiO217,1%, Al2O33.4 %Na2O
  2,4%, K2O 1,2%, ZrO20,1%, MgO 0,8%, Fe2O30,4%
[]
5. Kết quả phân loại:
[]
Tên thương mại: Ceramic Frit
[]
Tên gọi theo cấu tạo, công dụng: Men đã được
  Frit hóa dạng mảnh vỡ thủy tinh, thành phần gồm SiO251,2%, CaO
  23,4%, TiO217,1%, Al2O33.4% Na2O
  2,4%, K2O 1,2%, ZrO20,1%, MgO 0,8%, Fe2O30,4%
[]
Ký, mã hiệu, chủng loại: Không có thông
  tin            
  Nhà sản xuất: Không có thông tin
[]
thuộc nhóm3207"Thuốc màu đã
  pha chế, các chất cản quang đã pha chế và các loại màu

In [41]:
part1_4_index

3

In [42]:
part1_5_pattern = re.compile(r"5\.\s*Kết\s*quả")
for p_tag in part1:
    text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
    print(text)
    matches = part1_5_pattern.findall(text)  # Tìm tất cả các dãy số khớp với pattern trong văn bản
    print(matches)
    if matches:
        part1_5_index = part1.index(p_tag)
        print(part1_5_index)

1. Tên hàng theo khai báo: Phối liệu để
  nấu thủy tinh men Ceramic Frit UF- T39
[]
2. Đơn vị nhập khẩu:Công ty cổ phần
  Gạch men V.T.C; Địa chỉ: Lô 9, Khu công nghiệp Gò Dầu, Huyện Long Thành, Tỉnh
  Đồng Nai; Mã số thuế: 3600723398
[]
3. Số, ngày tờ khai hải quan:
  10010414600/A12 ngày 20/8/2014 tại Chi cục Hải quan Long Thành - Cục Hải quan
  Tỉnh Đồng Nai
[]
4, Tóm tắt mô tả và đặc tính hàng hóa:
  Men đã được Frit hóa dạng mảnh vỡ thủy tinh, thành phần gồm SiO251,2%, CaO 23,4%, TiO217,1%, Al2O33.4 %Na2O
  2,4%, K2O 1,2%, ZrO20,1%, MgO 0,8%, Fe2O30,4%
[]
5. Kết quả phân loại:
['5. Kết quả']
4
Tên thương mại: Ceramic Frit
[]
Tên gọi theo cấu tạo, công dụng: Men đã được
  Frit hóa dạng mảnh vỡ thủy tinh, thành phần gồm SiO251,2%, CaO
  23,4%, TiO217,1%, Al2O33.4% Na2O
  2,4%, K2O 1,2%, ZrO20,1%, MgO 0,8%, Fe2O30,4%
[]
Ký, mã hiệu, chủng loại: Không có thông
  tin            
  Nhà sản xuất: Không có thông tin
[]
thuộc nhóm3207"Thuốc màu đã
  pha chế, các chất cản quang đã pha chế v

In [43]:
part1_5_index

4

In [34]:
pattern = re.compile(r'\d{4}\s*\.\s*\d{2}\s*\.\s*\d{2}')
for p_tag in part1[part1_5_index:]:
    text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
    matches = pattern.findall(text)  # Tìm tất cả các dãy số khớp với pattern trong văn bản
    if matches:
        part_5_text = matches[-1]

In [35]:
part_5_text

'2841.30.00'